In [4]:
# Import standard packages
import os
import sys
import traceback
import multiprocessing
import numpy as np
from tqdm import tqdm
from langchain_ollama import ChatOllama
from datasets import load_dataset

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

/Users/jfeit/miniforge3/envs/phs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatOllama(
    model="llama3.2",
    temperature=0.5,
    n_gpu_layers=-1,
    n_threads=multiprocessing.cpu_count() - 1,
    verbose=False,
    seed=-1 # Random behavior so will not output the same thing always
)

In [37]:
def traverse_directory(directory):
    for root, _, files in os.walk(directory):
        for f in files:
            yield os.path.join(root, f)
            
file_paths = []
for f in traverse_directory("test_dirs"):
    file_paths.append(f)
    
str(file_paths)

"['test_dirs/images/dog.avif', 'test_dirs/images/cat.jpg', 'test_dirs/src/create_user.py', 'test_dirs/src/dogs.py', 'test_dirs/src/access_db.py']"

In [38]:
msg = llm.invoke('From the following list of files, which one(s) are most related to dogs? Return only the file paths and nothing else.' + str(file_paths))
msg.content

"'test_dirs/images/dog.avif', 'test_dirs/src/dogs.py'"

### KILT dataset

In [5]:
ds = load_dataset("facebook/kilt_tasks", "nq")

Generating test split: 100%|██████████| 1444/1444 [00:00<00:00, 350394.85 examples/s]


In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'meta', 'output'],
        num_rows: 87372
    })
    validation: Dataset({
        features: ['id', 'input', 'meta', 'output'],
        num_rows: 2837
    })
    test: Dataset({
        features: ['id', 'input', 'meta', 'output'],
        num_rows: 1444
    })
})